In [17]:
# Here we are following the instructions from the llama-recipes
# https://github.com/meta-llama/llama-recipes/blob/main/recipes/quickstart/Getting_to_know_Llama.ipynb

# The instructions are coupled with the VLLM docs here:
# https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html

In [20]:
# For setup we have followed following instructions

# virtualenv venv
# source venv/bin/activate
# pip install openai ipython jupyter

In [19]:
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="token-abc123",
)

In [3]:
def call(prompt):
    completion = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct",
    messages=[
        {"role": "user", "content": prompt}
    ]
    )
    return completion


In [6]:
prompt = "The typical color of a llama is: "
print(call(prompt).choices[0].message.content)

The typical color of a llama can vary, but common colors include:

1. Brown (light to dark)
2. Gray (light to dark)
3. White
4. Black
5. Red or reddish-brown

Llamas can also come in a variety of patterns, such as:

1. Solid (one color)
2. Piebald (white with patches of another color)
3. Appaloosa (spotted or mottled)
4. Roan (mixed with white hairs, giving a speckled appearance)

So, while there isn't just one "typical" color, brown, gray, and white are some of the most common colors found in llamas.


In [8]:
prompt = "The typical color of a llama is what? Answer in one word."
print(call(prompt).choices[0].message.content)

Beige.


In [9]:
prompt = "What is the average lifespan of a Llama? Answer the question in few words."
print(call(prompt).choices[0].message.content)

15-20 years typically.


In [10]:
prompt="What animal family are they? Answer the question in few words."
print(call(prompt).choices[0].message.content)

Depends on the animal.


In [12]:
# example of multi-turn chat, with storing previous context
prompt = """
User: What is the average lifespan of a Llama?
Assistant: 15-20 years.
User: What animal family are they?
"""
print(call(prompt).choices[0].message.content)

Llamas belong to the Camelid family, which also includes camels, alpacas, guanacos, and vicuñas. They are closely related to camels, but are native to South America.


In [13]:
prompt = '''
Llama started with 5 tennis balls. It buys 2 more cans of tennis balls. Each can has 3 tennis balls.
How many tennis balls does Llama have?

Answer in one word.
'''
print(call(prompt).choices[0].message.content)

Eleven.


In [15]:
# By default, Llama 3 models follow "Chain-Of-Thought" prompting
prompt = '''
Llama started with 5 tennis balls. It buys 2 more cans of tennis balls. Each can has 3 tennis balls.
How many tennis balls does Llama have?
'''
print(call(prompt).choices[0].message.content)

To find the total number of tennis balls Llama has, we'll start with the initial 5 tennis balls and then add the new ones.

Llama buys 2 cans of tennis balls, and each can has 3 tennis balls. So, 2 cans will have 2 x 3 = 6 tennis balls.

Now, let's add these new tennis balls to the initial ones:
5 (initial tennis balls) + 6 (new tennis balls) = 11

Llama now has 11 tennis balls.


In [16]:
prompt = """
15 of us want to go to a restaurant.
Two of them have cars
Each car can seat 5 people.
Two of us have motorcycles.
Each motorcycle can fit 2 people.
Can we all get to the restaurant by car or motorcycle?
Think step by step.
Provide the answer as a single yes/no answer first.
Then explain each intermediate step.
"""
print(call(prompt).choices[0].message.content)

Yes

Here are the intermediate steps:

1. **Number of people that can be seated in cars**: There are 2 cars, each seating 5 people. So, the total number of people that can be seated in cars is 2 * 5 = 10.

2. **Number of people that can be seated on motorcycles**: There are 2 motorcycles, each fitting 2 people. So, the total number of people that can be seated on motorcycles is 2 * 2 = 4.

3. **Total number of people that can be transported**: Adding the capacities of cars and motorcycles, we get 10 (from cars) + 4 (from motorcycles) = 14.

4. **Comparison with the total number of people**: There are 15 people in total who want to go to the restaurant. The total transportation capacity (14 people) is less than the number of people who want to go (15 people).

However, considering the owners of the cars and motorcycles, we have to account for them being part of the group as well. Since the question asks if "we all" can get to the restaurant by car or motorcycle, the implication is that 

In [1]:
!pip install -qU langchain langchain-community bs4 langchain-huggingface

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [281 lines of output]
      + /Users/suraj/code/work/llama-k8s/venv/bin/python /private/var/folders/ls/csbb474j2r9__hhcvmxq0rfh0000gn/T/pip-install-_6end3mq/numpy_c13257e7b69a438eb1be52d46564e099/vendored-meson/meson/meson.py setup /private/var/folders/ls/csbb474j2r9__hhcvmxq0rfh0000gn/T/pip-install-_6end3mq/numpy_c13257e7b69a438eb1be52d46564e099 /private/var/folders/ls/csbb474j2r9__hhcvmxq0rfh0000gn/T/pip-install-_6end3mq/numpy_c13257e7b69a438eb1be52d46564e099/.mesonpy-qfrpjc5e -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=/private/var/folders/ls/csbb474j2r9__hhcvmxq0rfh0000gn/T/pip-install-_6end3mq/numpy_c13257e7b69a438eb1be52d46564e099/.mesonpy-qfrpjc5e/meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: /private/var/folders/ls/csbb474j2r9__hhcvmxq0rfh0000gn/T/pip-install-_6end3mq/numpy_

In [27]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import bs4

# Step 1: Load the document from a web url
loader = WebBaseLoader(["https://huggingface.co/blog/llama31"])
documents = loader.load()

# Step 2: Split the document into chunks with a specified chunk size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(documents)

# Step 3: Store the document into a vector store with a specific embedding model
vectorstore = FAISS.from_documents(all_splits, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.